In [7]:
import pandas as pd

In [18]:
%pip install setfit

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
!git config --global user.email "ja_mahmahi@esi.dz"
!git config --global user.name "Anis Mahmahi"

In [6]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [31]:
import pandas as pd

test = pd.read_csv("/content/test-multi-label-oana.csv")
train = pd.read_csv("/content/train-multi-label-oana.csv")

In [32]:
train

,text,technique,article_name_x,article_name_y,Appeal_to_Authority,Appeal_to_fear-prejudice,Bandwagon,Black-and-White_Fallacy,Causal_Oversimplification,Doubt,...,Loaded_Language,"Name_Calling,Labeling","Obfuscation,Intentional_Vagueness,Confusion",Red_Herring,Reductio_ad_hitlerum,Repetition,Slogans,Straw_Men,Thought-terminating_Cliches,Whataboutism
0,Pamela Geller and Robert Spencer co-founded an...,Slogans,111111112,1.111111e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,"He added: ""We condemn all those whose behaviou...",Black-and-White_Fallacy,111111112,1.111111e+08,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Ms Geller, of the Atlas Shrugs blog, and Mr Sp...",Slogans,111111112,1.111111e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,On both of their blogs the pair called their b...,Loaded_Language,111111112,1.111111e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The UK should never become a stage for inflamm...,Flag-Waving,111111112,1.111111e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4142,"Comrades, these jokes you have been listening ...","Name_Calling,Labeling",999001970,1.998004e+09,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4143,In the future everyone will be the subject of ...,Loaded_Language,999001970,9.990020e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4144,Also the Left killed comedy.\n,"Exaggeration,Minimisation",999001970,1.998004e+09,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4145,That's what Columbia snowflakes thought was of...,"Name_Calling,Labeling",999001970,9.990020e+08,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
train['technique'] = train['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})
train.rename(columns={'Name_Calling,Labeling':'Name_Calling_Labeling'}, inplace=True)

test['technique'] = test['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})
test.rename(columns={'Name_Calling,Labeling':'Name_Calling_Labeling'}, inplace=True)


In [34]:
Discrediting_the_Opponent = ["Name_Calling_Labeling", "Whataboutism", "Appeal_to_fear-prejudice",
                             "Reductio_ad_hitlerum", "Doubt", "Causal_Oversimplification",]


In [39]:
train.technique.value_counts()

Name_Calling_Labeling        712
Doubt                        361
Appeal_to_fear-prejudice     158
Causal_Oversimplification    143
Whataboutism                  41
Reductio_ad_hitlerum          38
Name: technique, dtype: int64

In [36]:
train = train[train['technique'].isin(Discrediting_the_Opponent)]
test = test[test['technique'].isin(Discrediting_the_Opponent)]



In [37]:
columns_to_keep = ['text', 'technique']+Discrediting_the_Opponent
columns_to_keep

['text',
 'technique',
 'Name_Calling_Labeling',
 'Whataboutism',
 'Appeal_to_fear-prejudice',
 'Reductio_ad_hitlerum',
 'Doubt',
 'Causal_Oversimplification']

In [38]:
train = train.filter(items=columns_to_keep)
test = test.filter(items=columns_to_keep)


In [57]:
test.dtypes

text                         object
technique                    object
Name_Calling_Labeling         int64
Whataboutism                  int64
Appeal_to_fear-prejudice      int64
Reductio_ad_hitlerum          int64
Doubt                         int64
Causal_Oversimplification     int64
dtype: object

In [46]:
train.to_csv("train_yalowa.csv", index=False)
test.to_csv("test_yalowa.csv", index=False)

In [47]:
from datasets import load_dataset


dataset = load_dataset('csv', data_files={'train': "/content/train_yalowa.csv",'test': "/content/test_yalowa.csv"}, )
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'technique', 'Name_Calling_Labeling', 'Whataboutism', 'Appeal_to_fear-prejudice', 'Reductio_ad_hitlerum', 'Doubt', 'Causal_Oversimplification'],
        num_rows: 1453
    })
    test: Dataset({
        features: ['text', 'technique', 'Name_Calling_Labeling', 'Whataboutism', 'Appeal_to_fear-prejudice', 'Reductio_ad_hitlerum', 'Doubt', 'Causal_Oversimplification'],
        num_rows: 318
    })
})

In [48]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in Discrediting_the_Opponent]}


dataset = dataset.map(encode_labels)
dataset

Map:   0%|          | 0/1453 [00:00<?, ? examples/s]

Map:   0%|          | 0/318 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'technique', 'Name_Calling_Labeling', 'Whataboutism', 'Appeal_to_fear-prejudice', 'Reductio_ad_hitlerum', 'Doubt', 'Causal_Oversimplification', 'labels'],
        num_rows: 1453
    })
    test: Dataset({
        features: ['text', 'technique', 'Name_Calling_Labeling', 'Whataboutism', 'Appeal_to_fear-prejudice', 'Reductio_ad_hitlerum', 'Doubt', 'Causal_Oversimplification', 'labels'],
        num_rows: 318
    })
})

In [85]:
from setfit import SetFitModel
from sentence_transformers import SentenceTransformer

# model_body=SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")
# model_head=OneVsRestClassifier(LogisticRegression(class_weight="balanced")),
# model = SetFitModel(model_body=model_body, model_head=model_head,multi_target_strategy="one-vs-rest")



model = SetFitModel.from_pretrained('sentence-transformers/paraphrase-mpnet-base-v2', multi_target_strategy="one-vs-rest")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [86]:
model.to("cuda")

SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_head=OneVsRestClassifier(estimator=LogisticRegression()), multi_target_strategy='one-vs-rest', normalize_embeddings=False, labels=None, model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit with sentence-transformers/paraphrase-mpnet-base-v2', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id='sentence-transformers/paraphrase-mpnet-base-v2', hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metric_lines=[], widget=[], predict_example=None, label_example_list

In [87]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset

args = TrainingArguments(
    batch_size=16,
    num_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=10,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    column_mapping={"text": "text", "labels": "label"}  # Map dataset columns to text/label expected by trainer
)


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


In [88]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device

device(type='cuda')

In [89]:
# Train and evaluate
trainer.train()


***** Running training *****
  Num examples = 1817
  Num epochs = 2
  Total optimization steps = 3634
  Total train batch size = 16


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.235400,0.000011
2,No log,No log,0.226600,0.000000


  0%|          | 0/398 [00:00<?, ?it/s]

  0%|          | 0/398 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 3634.


In [90]:
metrics = trainer.evaluate(dataset['test'])
print(metrics)


Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.5849056603773585}


In [93]:
trainer.push_to_hub("anismahmahi/G2-multilabel-setfit-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/anismahmahi/G2-multilabel-setfit-model/commit/73535a9c68bddd798797cca6f4ba6038377cb6b2', commit_message='Add SetFit model', commit_description='', oid='73535a9c68bddd798797cca6f4ba6038377cb6b2', pr_url=None, pr_revision=None, pr_num=None)

In [91]:
from sklearn.metrics import f1_score,classification_report
preds = model.predict(list(test['text']))

In [ ]:
preds


In [81]:
test.technique.value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Name_Calling_Labeling        137
Appeal_to_fear-prejudice      81
Doubt                         51
Causal_Oversimplification     23
Whataboutism                  16
Reductio_ad_hitlerum          10
Name: technique, dtype: int64

In [82]:
len(dataset['test']['labels'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


318

In [84]:
test

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,technique,Name_Calling_Labeling,Whataboutism,Appeal_to_fear-prejudice,Reductio_ad_hitlerum,Doubt,Causal_Oversimplification
2,Geneva - The World Health Organisation chief o...,Doubt,0,0,0,0,1,0
4,He also pointed to the presence of the pneumon...,Appeal_to_fear-prejudice,0,0,1,0,0,0
5,He praised the rapid response from WHO and Mad...,Appeal_to_fear-prejudice,0,0,1,0,0,0
6,"""That means that Madagascar could be affected ...",Appeal_to_fear-prejudice,0,0,1,0,0,0
7,Members of the group and advocates say they fe...,Appeal_to_fear-prejudice,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
976,"The White House argued that Acosta’s behavior,...",Name_Calling_Labeling,1,0,0,0,0,0
980,After interacting with President Trump – Acost...,Name_Calling_Labeling,1,0,0,0,0,0
981,We look forward to continuing to defend the Wh...,Name_Calling_Labeling,1,0,0,0,0,0
983,The Acosta-CNN lawsuit was filed by Ted Boutro...,Name_Calling_Labeling,1,0,0,0,0,0


In [92]:
print(classification_report(dataset['test']['labels'],preds))

              precision    recall  f1-score   support

           0       0.83      0.74      0.79       148
           1       0.14      0.06      0.08        17
           2       0.77      0.74      0.75        89
           3       0.71      0.50      0.59        10
           4       0.64      0.70      0.67        56
           5       0.30      0.52      0.38        25

   micro avg       0.69      0.68      0.69       345
   macro avg       0.57      0.54      0.54       345
weighted avg       0.71      0.68      0.69       345
 samples avg       0.68      0.69      0.67       345



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
